# The detailed demonstration of the tropical rainfall diagnostic for ICON data:

# Calculation of histograms

### Content


1. [The load of packages, modules, and data](#1-load-of-packages-modules-and-data)

2. [Calculation of histogram of precipitation/tropical precipitation](#2-calculation-of-histogram-of-precipitationtropicalprecipitation)

    2.1. [Initializing the  diagnostic with the following attributes: num_of_bins, first_edge, width_of_bin](#21-initializing-the-class-object-with-the-following-attributes)

    2.2  [Mofification of tropical latitude band](#22-mofification-of-tropical-latitude-band)

    2.3. [Initialization of diagnostic with a selection of a time band](#23-initialization-of-diagnostic-with-a-selection-of-a-time-band)
    
    2.4. [Initialization of diagnostic with a logspaced bins](#24-initialization-of-diagnostic-with-a-logspaced-bins)

    2.5  [Calculation of histogram with weights (weights=reader.grid_area)](#25-calculation-of-histogram-with-weights-weightsreadergrid_area)
 

3. [Loading the histogram to/from storage](#3-loading-the-histogram-tofrom-storage)

[Go to the end of file](#the-end)

#

## 1. The load of packages, modules, and data

In [6]:
import sys
from aqua import Reader
sys.path.insert(0, '../')
from tropical_rainfall_class import TR_PR_Diagnostic as TR_PR_Diag

#### ICON data

In [7]:
reader = Reader(model="ICON", exp="ngc2009", source="atm_2d_ml_R02B09",  regrid="r100")
icon = reader.retrieve(regrid=True,  streaming=True, stream_step=3, stream_unit='days') 

## 2. Calculation of histogram of tropical precipitation


### 2.1. Initializing the class object with the following attributes: 
 - num_of_bins
 - first_edge
 - width_of_bin

In [8]:
diag = TR_PR_Diag(num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/20)

#### The tropical latitude band, by default, is the following

In [9]:
diag.trop_lat

10

In [10]:
hist_icon_trop  = diag.histogram(icon)
hist_icon_trop

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 8.579e+05 5.018e+04 ... 875.0
    frequency      (center_of_bin) float64 0.8326 0.04871 ... 0.0008493
    pdf            (center_of_bin) float64 1.665e+07 9.741e+05 ... 1.699e+04
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-06-14 16:19:50 regrid...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

### 2.2. Modification of tropical latitude band

##### You can modify the tropical latitude band as 

In [11]:
diag.trop_lat = 20

##### You can also modify the latitude band with the use of an argument of the histogram function 

In [13]:
hist_icon_trop  = diag.histogram(icon, trop_lat=30)
hist_icon_trop

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 2.711e+06 1.177e+05 ... 2.161e+03
    frequency      (center_of_bin) float64 0.8769 0.03807 ... 0.0007844 0.000699
    pdf            (center_of_bin) float64 1.754e+07 7.614e+05 ... 1.398e+04
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-06-14 16:19:50 regrid...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

### 2.3. Initialization of diagnostic with a selection of a time band 

##### You can specify `s_time` and `f_time` as integers. For example, 

In [14]:
diag = TR_PR_Diag( trop_lat = 30,  num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/15, s_time = 0, f_time = 47)

In [15]:
hist_icon = diag.histogram(icon)
hist_icon

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 3.333e-08 1e-07 ... 1.233e-06 1.3e-06
    width          (center_of_bin) float64 6.667e-08 6.667e-08 ... 6.667e-08
Data variables:
    counts         (center_of_bin) float64 2.914e+05 1.456e+04 ... 182.0 141.0
    frequency      (center_of_bin) float64 0.8634 0.04316 ... 0.0004178
    pdf            (center_of_bin) float64 1.295e+07 6.474e+05 ... 6.267e+03
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-06-14 16:19:50 regrid...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

##### Also, you can specify `s_time` and `f_time` as strings. For example, 

In [16]:
diag.s_time = '2020:01'
diag.f_time ='2020/03/20/12'

##### or

In [17]:
diag.s_time = '2020'
diag.f_time ='2020.03.20'

##### There is the possibility of specifying only the year band or only the months' band. For example, we can select June, July, and August in a whole dataset as

In [18]:
diag.s_month = 3
diag.f_month = 6 

### 2.4.  Initialization of diagnostic with a log spaced bins

In [19]:
bins = [1.00000000e-09, 1.63789371e-09, 2.68269580e-09, 4.39397056e-09,
       7.19685673e-09, 1.17876863e-08, 1.93069773e-08, 3.16227766e-08,
       5.17947468e-08, 8.48342898e-08, 1.38949549e-07, 2.27584593e-07,
       3.72759372e-07, 6.10540230e-07, 1.00000000e-06]

In [20]:
diag = TR_PR_Diag(bins = bins)

In [21]:
hist_icon_trop_logspaced  = diag.histogram(icon)
hist_icon_trop_logspaced

<xarray.Dataset>
Dimensions:        (center_of_bin: 14)
Coordinates:
  * center_of_bin  (center_of_bin) float64 1.319e-09 2.16e-09 ... 8.053e-07
    width          (center_of_bin) float64 6.379e-10 1.045e-09 ... 3.895e-07
Data variables:
    counts         (center_of_bin) int64 26050 28164 30182 ... 30324 22300 12104
    frequency      (center_of_bin) float64 0.06091 0.06586 ... 0.05214 0.0283
    pdf            (center_of_bin) float64 9.549e+07 6.303e+07 ... 7.267e+04
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-06-14 16:19:50 regrid...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

### 2.5. Calculation of histogram with weights (weights=reader.grid_area)

In [22]:
diag = TR_PR_Diag(num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/15, s_time = 0, f_time = 47)

In [23]:
hist_icon_trop_weighted  = diag.histogram(icon, weights=reader.grid_area)
hist_icon_trop_weighted

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 3.333e-08 1e-07 ... 1.233e-06 1.3e-06
    width          (center_of_bin) float64 6.667e-08 6.667e-08 ... 6.667e-08
Data variables:
    counts         (center_of_bin) float64 7.36e+13 4.932e+12 ... 3.696e+10
    frequency      (center_of_bin) float64 0.8327 0.0558 ... 0.0006977 0.0004181
    pdf            (center_of_bin) float64 1.249e+07 8.37e+05 ... 6.272e+03
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-06-14 16:19:50 regrid...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

##### You can also parse only the `bins` attribute of the class  instead of the specification of `num_of_bins`, `first_edge`, and `width_of_bin`

In [24]:
diag = TR_PR_Diag(bins = bins)

In [25]:
hist_icon_trop_weighted  = diag.histogram(icon, weights=reader.grid_area)
hist_icon_trop_weighted

<xarray.Dataset>
Dimensions:        (center_of_bin: 14)
Coordinates:
  * center_of_bin  (center_of_bin) float64 1.319e-09 2.16e-09 ... 8.053e-07
    width          (center_of_bin) float64 6.379e-10 1.045e-09 ... 3.895e-07
Data variables:
    counts         (center_of_bin) float64 2.423e+12 2.866e+12 ... 7.722e+11
    frequency      (center_of_bin) float64 0.06147 0.0727 ... 0.04079 0.01959
    pdf            (center_of_bin) float64 9.636e+07 6.958e+07 ... 5.029e+04
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-06-14 16:19:50 regrid...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

#

## 3. Loading the histogram to/from storage

##### Saving the histogram in storage. Notice: the name of the file will be unique and depends on the time band 

In [26]:
path_to_save='/work/bb1153/b382267/AQUA/histograms/'
diag.save_histogram(dataset=hist_icon, path_to_save=path_to_save, name_of_file='icon')

'/work/bb1153/b382267/AQUA/histograms/icon_2020-01-20T00_2020-01-20T23_histogram.pkl'

##### Loading/opening the histogram from the storage 

In [27]:
diag.load_histogram(path_to_dataset='/work/bb1153/b382267/AQUA/histograms/icon_2020-01-20T00_2020-01-20T23_histogram.pkl')

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 3.333e-08 1e-07 ... 1.233e-06 1.3e-06
    width          (center_of_bin) float64 6.667e-08 6.667e-08 ... 6.667e-08
Data variables:
    counts         (center_of_bin) float64 2.914e+05 1.456e+04 ... 182.0 141.0
    frequency      (center_of_bin) float64 0.8634 0.04316 ... 0.0004178
    pdf            (center_of_bin) float64 1.295e+07 6.474e+05 ... 6.267e+03
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-06-14 16:19:50 regrid...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

[Back to the top of file](#structure)

#

##### The end